<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We import libraries for data handling, optimization, portfolio simulation, and plotting, along with some support for warnings and date management.

In [ ]:
import os
import warnings
from datetime import datetime
import riskfolio as rp

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from vectorbt.portfolio.enums import Direction, SizeType
from vectorbt.portfolio.nb import order_nb, sort_call_seq_nb

In [ ]:
vbt.settings.returns["year_freq"] = "252 days"

In [ ]:
warnings.filterwarnings("ignore")

This segment brings in the core libraries needed to build, test, and evaluate a portfolio strategy. We're setting general behavior like treating each year as 252 trading days and turning off warning messages to keep the output tidy. With these imports, we enable data downloading, calculations, optimization, and visualization in later steps.

## Load and prepare historical data

We select major finance sector stocks, then download their historical prices and clean up the dataset by removing incomplete dates.

In [ ]:
tickers = [
"JPM", "V", "MA", "BAC", "WFC", "GS", "MS", "AXP", "C"
]

In [ ]:
data = yf.download(tickers, start="2010-01-01", end="2024-06-30", auto_adjust=False)["Close"].dropna()

Here we define our investment universe by listing ticker symbols for several well-known financial companies. We then collect daily closing prices for these stocks from Yahoo Finance, covering more than a decade. Removing dates with missing values leaves us with a consistent dataset for robust backtesting.

## Define portfolio simulation functions

We set how often to rebalance, how to look back at price history for optimization, and how to place portfolio trades.

In [ ]:
num_tests = 2000
ann_factor = data.vbt.returns(freq="D").ann_factor

In [ ]:
def pre_sim_func_nb(sc, every_nth):
    sc.segment_mask[:, :] = False
    sc.segment_mask[every_nth::every_nth, :] = True
    return ()

In [ ]:
def pre_segment_func_nb(
    sc, find_weights_nb, history_len, ann_factor, num_tests, srb_sharpe
):
    if history_len == -1:
        close = sc.close[: sc.i, sc.from_col : sc.to_col]
    else:
        if sc.i - history_len <= 0:
            return (np.full(sc.group_len, np.nan),)
        close = sc.close[sc.i - history_len : sc.i, sc.from_col : sc.to_col]

    best_sharpe_ratio, weights = find_weights_nb(sc, close, num_tests)
    srb_sharpe[sc.i] = best_sharpe_ratio

    size_type = np.full(sc.group_len, SizeType.TargetPercent)
    direction = np.full(sc.group_len, Direction.LongOnly)
    temp_float_arr = np.empty(sc.group_len, dtype=np.float_)
    for k in range(sc.group_len):
        col = sc.from_col + k
        sc.last_val_price[col] = sc.close[sc.i, col]
    sort_call_seq_nb(sc, weights, size_type, direction, temp_float_arr)

    return (weights,)

In [ ]:
def order_func_nb(oc, weights):
    col_i = oc.call_seq_now[oc.call_idx]
    return order_nb(
        weights[col_i],
        oc.close[oc.i, oc.col],
        size_type=SizeType.TargetPercent,
    )

This block sets the groundwork for our portfolio simulation. We specify how often rebalancing happens, how much historical data to use when making allocation decisions, and how orders are created during simulation. The functions handle date logic, extract relevant price data, optimize for the best risk-adjusted returns, and structure trades to aim for those weights. Using this setup, our strategy can dynamically recalculate allocations throughout the backtest.

## Optimize portfolio weights and run backtest

We run an optimization to find weights that maximize risk-adjusted returns using past data, then simulate the portfolio and review performance.

In [ ]:
def opt_weights(sc, close, num_tests):
    close = pd.DataFrame(close, columns=tickers)
    returns = close.pct_change().dropna()
    port = rp.Portfolio(returns=returns)
    port.assets_stats(method_mu="hist", method_cov="hist")
    w = port.optimization(model="Classic", rm="CDaR", obj="Sharpe", hist=True)
    weights = np.ravel(w.to_numpy())
    shp = rp.Sharpe(w, port.mu, cov=port.cov, returns=returns, rm="CDaR", alpha=0.05)
    return shp, weights

In [ ]:
sharpe = np.full(data.shape, np.nan)
pf = vbt.Portfolio.from_order_func(
    data,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(opt_weights, 252 * 4, ann_factor, num_tests, sharpe),
    cash_sharing=True,
    group_by=True,
    use_numba=False,
    freq="D"
)

In [ ]:
pf.plot_cum_returns()

In [ ]:
pf.stats()

In [ ]:
bm_returns = pf.benchmark_returns()
bm_returns_acc = bm_returns.vbt.returns(
    freq="1d",
    year_freq="252 days",
)
print(f"Benchmark sharpe: {bm_returns_acc.sharpe_ratio()}")
print(f"Benchmark drawdown: {bm_returns_acc.max_drawdown()}")

We define how to select our portfolio weights by maximizing the Sharpe ratio—this balances return against risk. Then we use a simulation engine to run our strategy, regularly rebalancing according to these optimal allocations. Finally, we visualize cumulative returns, print statistics, and compare results against a simple buy-and-hold approach. This gives us clear insight into how our adaptive strategy performed over many years, using real historical price data.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.